In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import sys
import os

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *

import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes/'

In [9]:
###### reconstruct thumbnail volume  #######

stack = 'MD585'

downsample_factor = 32

section_thickness = 20 # in um
xy_pixel_distance_lossless = 0.46
xy_pixel_distance_tb = xy_pixel_distance_lossless * 32 # in um, thumbnail
# factor = section_thickness/xy_pixel_distance_lossless

xy_pixel_distance_downsampled = xy_pixel_distance_lossless * downsample_factor
z_xy_ratio_downsampled = section_thickness / xy_pixel_distance_downsampled

dm = DataManager(stack=stack)

volume = np.zeros((dm.image_height/downsample_factor, 
                   dm.image_width/downsample_factor, 
                   int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.uint8)

# volume = np.zeros((1500, 2000, int(section_number_lookup[stack]*z_xy_ratio_downsampled)), np.uint8)

section_bs_begin, section_bs_end = section_range_lookup[stack]
print section_bs_begin, section_bs_end

for sec in range(section_bs_begin, section_bs_end+1):
    
    print sec
    
#     dm.set_slice(section=sec)

    im = img_as_ubyte(rgb2gray(imread(os.environ['DATA_DIR'] + '/%(stack)s_thumbnail_aligned_mask_cropped/%(stack)s_%(slice)04d_thumbnail_aligned_mask_cropped.png' \
                        % {'stack': stack, 'slice': sec})))

#     im = img_as_ubyte(imread(os.environ['DATA_DIR'] + '/%(stack)s_thumbnail_aligned_masked/%(stack)s_%(slice)04d_thumbnail_aligned_masked.png' \
#         % {'stack': stack, 'slice': sec})) 
    
    volume[:,:, int(z_xy_ratio_downsampled*sec):int(z_xy_ratio_downsampled*(sec+1))] = im[..., np.newaxis]

78 347
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331

In [10]:
volume.shape

(408, 445, 597)

In [13]:
bp.pack_ndarray_file(volume, volume_dir + '%(stack)s/%(stack)s_thumbnailVolume.bp' % {'stack': stack})

In [ ]:
# volume_dir = '/oasis/projects/nsf/csd395/yuncong/CSHL_volumes'

volume[volume < 1e-6] = 0
xs, ys, zs = np.where(volume > 0)

xmin = xs.min()
xmax = xs.max()
ymin = ys.min()
ymax = ys.max()
zmin = zs.min()
zmax = zs.max()

print xmin, xmax
print ymin, ymax
print zmin, zmax

In [ ]:
volume = volume[xmin:xmax+1, ymin:ymax+1, zmin:zmax+1]

In [ ]:
dm = DataManager(stack=stack)

theta_z = 0
theta_y = 0
theta_x = 0
tx = 0
ty = 0
tz = 0
skew_z = 0
skew_y = 0
skew_x = 0

# cos_z = np.cos(theta_z)
# sin_z = np.sin(theta_z)
# cos_y = np.cos(theta_y)
# sin_y = np.sin(theta_y)
# Rz = np.array([[cos_z, -sin_z, 0], [sin_z, cos_z, 0], [0,0,1]])
# Ry = np.array([[cos_y, 0, -sin_y], [0, 1, 0], [sin_y, 0, cos_y]])
# Rx = np.array([[1, 0, 0], [0, cos_y, -sin_y], [0, sin_y, cos_y]])
# R = np.dot(Rz, Ry, Rx)


In [ ]:
xs, ys, zs = np.meshgrid(range(volume.shape[0]), 
                         range(volume.shape[1]), 
                         range(volume.shape[2]))

In [ ]:
xys = np.c_[xs.flat, ys.flat, zs.flat]

In [ ]:
# manual affine correction
T = np.eye(3)
T[0,2] = .1 # + move right side down
T[2,0] = -.1 # + move bottom side left
# T[2,0] = .1

print T

In [ ]:
xyzs_new = np.dot(T, np.c_[xs.flat, ys.flat, zs.flat].T).astype(np.int).T

In [ ]:
valid = (xyzs_new[:,0] >= 0) & (xyzs_new[:,1] >= 0) & (xyzs_new[:,2] >= 0) &\
(xyzs_new[:,0] < volume.shape[0]) & (xyzs_new[:,1] < volume.shape[1]) & (xyzs_new[:,2] < volume.shape[2])

In [ ]:
# volume_new = np.zeros((dm.image_height/downsample_factor, 
#                    dm.image_width/downsample_factor, 
#                    int(n_sec[stack]*z_xy_ratio_downsampled)), np.uint8)

volume_new = np.zeros_like(volume, np.uint8)

volume_new[xyzs_new[valid, 0], xyzs_new[valid, 1], xyzs_new[valid, 2]] = \
volume[xys[valid, 0], xys[valid, 1], xys[valid, 2]]

In [ ]:
for y in range(0, volume.shape[1], 50):
    plt.imshow(volume_new[:, y, :], cmap=plt.cm.gray);
    plt.title('y = %d' % y);
    plt.show();

In [ ]:
for y in range(0, volume.shape[1], 10):
    plt.imshow(volume[:, y, :], cmap=plt.cm.gray);
    plt.title('y = %d' % y);
    plt.show();